In [1]:
import Pkg;Pkg.activate("convex")
Pkg.instantiate()
# Pkg.resolve()
# Pkg.gc()
# Pkg.precompile()

  Activating environment at `~/SageMaker/networks_hub/SAN/code/network_simulations/convex/Project.toml`
  No Changes to `~/SageMaker/networks_hub/SAN/code/network_simulations/convex/Project.toml`
  No Changes to `~/SageMaker/networks_hub/SAN/code/network_simulations/convex/Manifest.toml`
      Active manifest files: 2 found
      Active artifact files: 80 found
      Active scratchspaces: 6 found
     Deleted no artifacts, repos, packages or scratchspaces


In [2]:
using LinearAlgebra, DataFrames, XLSX, Missings, Test, SpecialFunctions,  SparseArrays , Random
#using ProximalOperators, SCS, SequentialConvexRelaxation, Convex, COSMO
using JuMP, Ipopt, DistributionsAD, Quadrature
#using JuMP, Ipopt,  Distributions, DistributionsAD, SpecialFunctions, NLsolve, Random
#using Quadrature, Cubature, Cuba, QuadGK
#using EAGO, Alpine, Juniper, NLopt, Pavito, SCIP, GLPK

In [3]:
## load data

xf = XLSX.readxlsx("node_stats_forsimulation_all.xlsx") 
data = vcat( [(XLSX.eachtablerow(xf[s]) |> DataFrames.DataFrame) for s in XLSX.sheetnames(xf)]... ) #for s in XLSX.sheetnames(xf) if (s!="Aggregates Composition" && s!="Dealer Aggregates" && s!="Approx Aggregates")
unique!(data) # delete duplicate rows, use `nonunique(data)` to see if there are any duplicates
data = data[isequal.(data.qt_dt,195), :] # keep quarter == 195 = 2008q4
sort!(data, :assets, rev = true)
units = 1e6;
data[:,[:w, :c, :assets, :p_bar, :b]] .= data[!,[:w, :c, :assets, :p_bar, :b]]./units
# data.b[:] .= missing
# data.c[:] .= missing

col_with_miss = names(data)[[any(ismissing.(col)) for col = eachcol(data)]] # columns with at least one missing
data_nm = coalesce.(data, data.assets/1.5) # replace missing by a value
nm_c = findall(x->x==0,ismissing.(data.c))
nm_b = findall(x->x==0,ismissing.(data.b))
dropmissing(data, [:delta, :delta_alt, :w, :assets, :p_bar]) # remove type missing

names(data) # column names
describe(data)
show(data, allcols = true)

867×12 DataFrame
 Row │ nm_short                      qt_dt  tkr   delta      delta_alt    beta      w          c         assets    nvi_benchmark  p_bar     b        
     │ Any                           Any    Any   Any        Any          Any       Any        Any       Any       Any            Any       Any      
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ JPMORGAN CHASE & CO           195    JPM   0.0106652  0.00862097   0.56278   0.168059   1.45164   2.17505   11.0931        2.00699   0.877498
   2 │ CITIGROUP                     195    C     0.0647046  0.0475393    0.481591  0.144022   1.38565   1.93847   11.0931        1.79445   0.930258
   3 │ BANK OF AMER CORP             195    BAC   0.016278   0.0117393    0.50803   0.17871    1.51681   1.82207   11.0931        1.64336   0.808482
   4 │ WELLS FARGO & CO              195    WFC   0.0047069  0.00462975   0.397362  0.

In [19]:
N=30
T = Float64
temp = Array{T}(data[1:N,[:delta, :delta_alt, :w, :assets, :p_bar]])
delta = copy(temp[:,1]); 
delta_alt = copy(temp[:,2]);
w = copy(temp[:,3]); 
assets= copy(temp[:,4]);
p_bar = copy(temp[:,5]);

γ = T(0)
M = N^2+3*N

rng =  Random.seed!(123)
x = ones(T,N)/10 #rand(T, N)
Ex = ones(T,N)/10 #ones(T,N)/2 # expectation of x

# upper and lower bounds
# low<=z<=upp
low = spzeros(T,M)
upp = vcat(ones(T,N^2),p_bar,assets,p_bar)

# linear constraints
H₁ = spzeros(T,M,M)
h₁₁ᵀ = hcat(sparse(kron(Matrix{T}(I,N,N),p_bar')))
h₁ᵀ = hcat(h₁₁ᵀ,spzeros(T,N,N),sparse(I,N,N),spzeros(T,N,N))
q₁ = assets

H₂ = spzeros(T,M,M)
h₂₁ᵀ = repeat(spdiagm(p_bar),1,N)
h₂ᵀ = hcat(h₂₁ᵀ,sparse(I,N,N),spzeros(T,N,N),spzeros(T,N,N))
q₂ = p_bar

# hᵀz=q
hᵀ = vcat(h₁ᵀ,h₂ᵀ)
q = vcat(q₁,q₂)

# quadratic constraints
li = LinearIndices((N,N))
liᵀ= transpose(li)
id = li[tril!(trues(N,N), 0)]
idᵀ = liᵀ[tril!(trues(N,N), 0)]
H₃= sparse(vcat(id,idᵀ),vcat(idᵀ,id),T(0.5),M,M)
h₃ = spzeros(T,M)
q₃ = spzeros(T,1)

# zᵀHz = 0
H = H₃

# nonlinear constraints
# [h₄₄ᵀ[i]z=min{p_bar[i],max{zᵀH₄[i]z+h₄ᵀ[i]z+q₄[i],0}} for i=1,2,...,N]
# note: h₄ᵀ[i] depends on x
H₄  = [sparse( vcat(N*(i-1)+1:N*i,M-N+1:M), vcat(M-N+1:M,N*(i-1)+1:N*i), (1+γ)*ones(T,2*N)/2,M,M) for i=1:N] # H₄⁽ⁱ⁾=H₄[i]
h₄ᵀ = [sparse([1],[N^2+N+i],(1+γ)*(T(1)-x[i]),1,M) for i=1:N] # h₄ᵀ⁽ⁱ⁾=h₄ᵀ[i]
h₄₄ᵀ= [sparse([1],[N^2+2*N+i],T(1),1,M) for i=1:N] # h₄₄⁽ⁱ⁾ᵀ=h₄₄ᵀ[i]
q₄ = -γ*p_bar # q₄⁽ⁱ⁾=q₄⁽ⁱ⁾[i]
# or, if we ignore the min{p_bar,max{...,0}}, we get the quadratic constraint
# h₄₄ᵀ[i]z=zᵀH₄[i]z+h₄ᵀ[i]z+q₄[i] for i=1,2,...,N
# which we can also write as
# transpose(z)*H₄[i]*z==(h₄₄ᵀ[i] - h₄ᵀ[i])*z-q₄[i]  for i=1,2,...,N
# with h₄₄₄ᵀ[i] == h₄₄ᵀ[i] - h₄ᵀ[i]  for i=1,2,...,N
h₄₄₄ᵀ = [h₄₄ᵀ[i] - h₄ᵀ[i] for i=1:N]

# chance   constraints
# Prob(h₅ᵀ[i]z>=q₅[i])=delta[i]
h₅ᵀmat = hcat(spzeros(T,N,N^2),spzeros(T,N,N),sparse(I,N,N),spzeros(T,N,N))
h₅ᵀ= [h₅ᵀmat[i, :] for i in 1:N]
q₅ = w

# objective function
# max or min E[h₀ᵀz]
h₀₃ᵀ= transpose(Ex)
h₀₄ᵀ=-ones(T,1,N)
h₀ᵀ = hcat(spzeros(T,1,N^2),spzeros(T,1,N),h₀₃ᵀ,h₀₄ᵀ)

# spy(H₀)
# spy(H₀[1:N^2,1:N^2])

# convert from sparse into full arrays
lowA=Array(low)
uppA=Array(upp)
hᵀA=Array(hᵀ)
qA=Array(q)
HA=Array(H)
h₄₄ᵀA = Array.(h₄₄ᵀ)
h₄₄₄ᵀA = Array.(h₄₄₄ᵀ)
H₄A=Array.(H₄)
h₄ᵀA=Array.(h₄ᵀ)
q₄A=Array(q₄)
h₅ᵀA=Array.(h₅ᵀ)
q₅A=Array(q₅)
h₀ᵀA=Array(h₀ᵀ)

1×990 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0

In [20]:
# unit testing dist_cdf
# function dist_cdf(p...)
#     #α,β,wc = p[1], p[2], p[3]
#     obj(x,p) = (x>=p[3])*DistributionsAD.pdf(DistributionsAD.Beta(p[1],p[2]),x)
#     return Quad1d(obj,p)
# end
# function Quad1d(obj,p)
#     prob = QuadratureProblem(obj,0,1,p,order=9)
#     return solve(prob,QuadGKJL(),reltol = 1e-9)[1]
# end
# p=[2.23,4.15,0.11]
# @test dist_cdf(p...) ≈ DistributionsAD.ccdf(DistributionsAD.Beta(p[1],p[2]),p[3])

In [ ]:
m = Model(optimizer_with_attributes(Ipopt.Optimizer,"tol"=>1e-5,"max_iter"=>100000,"print_level"=>7,"check_derivatives_for_naninf"=>"yes")) #,"linear_solver"=>"pardiso"

@variable(m, lowA[i]<=z[i=1:M]<=uppA[i]) 
@variable(m, 1.0<=α[i=1:N]<=2.0, start = 4.1) 
@variable(m, 1.0<=β[i=1:N]<=150.0, start = 30.0) 
@constraint(m, hᵀA*z.==qA) 
@constraint(m,transpose(z)*HA*z==0.0) 
@constraint(m,px[i=1:N],transpose(z)*H₄A[i]*z.==h₄₄₄ᵀA[i]*z.- q₄A[i] )
@constraint(m,z[diag(LinearIndices(ones(N,N)))].==0)

# match probabilities of default
function dist_cdf(p...)
    #α,β,wc = p[1], p[2], p[3]
    obj(x,p) = (x>=p[3])*DistributionsAD.pdf(DistributionsAD.Beta(p[1],p[2]),x)
    return Quad1d(obj,p)
end
function Quad1d(obj,p)
    prob = QuadratureProblem(obj,0,1,p,order=5)
    return Quadrature.solve(prob,QuadGKJL(),reltol = 1e-5)[1]
end

JuMP.register(m, :dist_cdf, 3, dist_cdf; autodiff=true)
@NLexpression(m,wc[i=1:N],w[i]/z[N^2+N+i]) #c[i] = z[N^2+N+i]
for i in 1:N
    @eval ($(Symbol("vv$i"))) = [α[$i],β[$i],wc[$i]]
    @eval @NLconstraint(m,dist_cdf(($(Symbol("vv$i")))...)== delta[$i]) 
end

ex = @expression(m,(h₀ᵀA*z)[1])
@objective(m, Max,  ex )

unset_silent(m)
JuMP.optimize!(m)

In [27]:
st = termination_status(m)
objSol = objective_value(m)

zsol = JuMP.value.(z)
Asol = reshape(zsol[1:N^2],N,N)
bsol = zsol[N^2+1:N^2+N] 
csol = zsol[N^2+N+1:N^2+2*N]
psol = zsol[N^2+2*N+1:N^2+3*N]

trueObj = objSol + sum(Ex.*csol) + sum(p_bar)
@show st, objSol, trueObj

tol = 1e-5
@testset "check solution" begin
    @test norm( sum(Asol,dims=2).* p_bar .- (p_bar .- bsol)) < tol
    @test norm( Asol' * p_bar .- (assets .- csol)) < tol
    @test norm(diag(Asol)) < tol
    @test norm([Asol[i,j]*Asol[j,i] for i=1:N , j=1:N]) < tol
    @test all(-tol .<=Asol.<=1+tol)
    @test all(-tol .<=bsol.<=p_bar .+ tol)
    @test all(-tol .<=csol.<=assets .+ tol)   
    @test all(-tol .<=psol.<=p_bar .+ tol)   
    @test norm(min.(p_bar, max.((1 .+γ).*(Asol'*psol .+ csol .- x.*csol) .- γ.*p_bar,0)).-psol)<tol

end

@show L_opt = sum(p_bar)+objSol+sum(x.*csol)
@show L_d = sum(x.*csol) + max.(0,x.-w)[1] # disconnected network

β = (p_bar.-bsol)./p_bar # financial connectivity: proportion of liabilities inside the network
β⁺ = maximum(β)

@show (L_opt,L_d)
ratio = L_opt/L_d
bound = 1 + sum(delta.*csol)/((1-β⁺)*sum(csol))
@show (ratio,bound)

(st, objSol, trueObj) = (MathOptInterface.NUMERICAL_ERROR, -12.78942094865548, 2.899039564459583)
Test Summary:  | Pass  Total
check solution |    9      9
L_opt = sum(p_bar) + objSol + sum(x .* csol) = 2.8990395644595828
L_d = sum(x .* csol) + max.(0, x .- w)[1] = 1.3303376213025646
(L_opt, L_d) = (2.8990395644595828, 1.3303376213025646)
(ratio, bound) = (2.179175810739732, 1.214419869495001)


(2.179175810739732, 1.214419869495001)

In [ ]:
# Pkg.add("FastGaussQuadrature")
using FastGaussQuadrature
α=1.3
β=4.2
a = β-1
b = α-1

num_nodes = 500000 # 100000, 1000, 10
nodes, weights = gaussjacobi(num_nodes, a, b); 

#cdf 
f(x,wc) = (x .> wc)
f_tilde(x,wc)=f(1/2+x/2,wc)
F(wc) = dot(weights,f_tilde.(nodes,wc))/beta(α,β)/2^(α+β-1)
F_exact(wc) = DistributionsAD.ccdf(DistributionsAD.Beta(α,β),wc)
for wc = 0.0:0.01:1.0
    @test abs(F(wc) - F_exact(wc))<1e-5
end

In [ ]:
function beta_ccdf(p)
    α,β,x = p[1],p[2],p[3]
    a = β-1
    b = α-1
    num_nodes = 4000 
    nodes, weights = gaussjacobi(num_nodes, a, b); 

    #cdf 
    return dot(weights,( (1/2 .+ nodes./2) .> x) )/beta(α,β)/2^(α+β-1)
end

In [ ]:
using ForwardDiff,Zygote

In [ ]:
α=1.3
β=6.0
wc=0.5
@show beta(α,β)/2^(α+β-1)
@show beta_ccdf(α,β,wc)
@show DistributionsAD.ccdf(DistributionsAD.Beta(α,β),wc)

In [ ]:
using Quadrature, DistributionsAD
using BenchmarkTools

In [ ]:
function dist_cdf1(p...)
    #α,β,wc = p[1], p[2], p[3]
    obj(x,p) = (x>=p[3])*DistributionsAD.pdf(DistributionsAD.Beta(p[1],p[2]),x)
#     prob = QuadratureProblem(obj,0.0,1.0,p,order=2)
#     Quadrature.solve(prob,QuadGKJL(),reltol = 1e-2)[1]
    return Quad1d1(obj,p)
end
function Quad1d1(obj,p)
    prob = QuadratureProblem(obj,0,1,p,order=2)
    return solve(prob,QuadGKJL(),reltol = 1e-2)[1]
end
p=[2.23,4.15,0.51]
@btime dist_cdf1($p...)
@code_warntype dist_cdf1(p...)

In [ ]:
dist_cdf3(p...)::Real=solve(QuadratureProblem((x,p)->(x>=p[3])*DistributionsAD.pdf(DistributionsAD.Beta(p[1],p[2]),x),0,1,p,order=2),QuadGKJL(),reltol = 1e-2)[1]

p=[2.23,4.15,0.51]
@btime dist_cdf3($p...)
@code_warntype dist_cdf3(p...)

In [ ]:
dist_cdf4(p...)::Real=Quadrature.solve(Quadrature.QuadratureProblem((x,p)->(x>=p[3])*DistributionsAD.pdf(DistributionsAD.Beta(p[1],p[2]),x),0,1,p,order=2),QuadGKJL(),reltol = 1e-2)[1]


p=[1.0,3.0,0.2]
@btime dist_cdf4($p...)
@code_warntype dist_cdf4(p...)

In [ ]:
@fastmath begin
    
end

In [ ]:
56.448 μs (153 allocations: 5.47 KiB)

In [ ]:
p=[α,β,0.41]
beta_ccdf(p)

In [ ]:
dp1 = Zygote.gradient(beta_ccdf,p)

In [ ]:
dp2 = FiniteDiff.finite_difference_gradient(beta_ccdf,p)
dp3 = ForwardDiff.gradient(beta_ccdf,p)
dp1[1] ≈ dp2 ≈ dp3

In [ ]:
# Pkg.add("FastGaussQuadrature")
#using FastGaussQuadrature
α=1.3
β=100.2
a = β-1
b = α-1

num_nodes = 4000 # 100000, 1000, 10
nodes, weights = gaussjacobi(num_nodes, a, b); 

#cdf 
f(x,wc) = (x .> wc)
f_tilde(x,wc)=f(1/2+x/2,wc)
F(wc) = dot(weights,f_tilde.(nodes,wc))/beta(α,β)/2^(α+β-1)
F_exact(wc) = DistributionsAD.ccdf(DistributionsAD.Beta(α,β),wc)

# mean
f(x) = x
f_tilde(x)=f(1/2+x/2)
Ex = dot(weights,f_tilde.(nodes))/beta(α,β)/2^(α+β-1)
Ex_exact = α/(α+β)

# var
f(x) = (x-Ex)^2
f_tilde(x)=f(1/2+x/2)
Varx = dot(weights,f_tilde.(nodes))/beta(α,β)/2^(α+β-1)
Varx_exact = α*β/((α+β)^2*(α+β+1))

# E[log(X)]
f(x) = log(x)
f_tilde(x)=f(1/2+x/2)
Elogx = dot(weights,f_tilde.(nodes))/beta(α,β)/2^(α+β-1)
Elogx_exact = digamma(α)-digamma(α+β)

# E[X log(X)]
f(x) = x*log(x)
f_tilde(x)=f(1/2+x/2)
Exlogx = dot(weights,f_tilde.(nodes))/beta(α,β)/2^(α+β-1)
Exlogx_exact = (α/(α+β))*(digamma(α+1)-digamma(α+β+1))

# var[log(X)]
f(x) = (log(x)-Elogx)^2
f_tilde(x)=f(1/2+x/2)
varxlogx = dot(weights,f_tilde.(nodes))/beta(α,β)/2^(α+β-1)
varxlogx_exact = trigamma(α)-trigamma(α+β)

@testset "beta quadrature" begin
    @test Ex ≈ Ex_exact
    @test Varx ≈ Varx_exact
    @test Elogx ≈ Elogx_exact
    @test Exlogx ≈ Exlogx_exact    
    @test varxlogx ≈ varxlogx_exact
end

# next step: create a function that integrates between w/c and 1
# using change of variables rather than indicator function in the integrand
# then use in constraint delta == integral(...)

In [ ]:
x
w

In [ ]:
using Quadrature
using ForwardDiff

In [ ]:
function dist_cdf(p...)
    #α,β,wc = p[1], p[2], p[3]
    obj(x,p) = DistributionsAD.pdf(DistributionsAD.Beta(p[1],p[2]),x[1])
    prob = QuadratureProblem(obj,p[3],1.0,p[1:2])
    Quadrature.solve(prob,QuadGKJL(),reltol=1e-2,abstol=1e-2)[1]
end
#@show DistributionsAD.ccdf(DistributionsAD.Beta(1.1,2.3),0.01233)
ForwardDiff.gradient(dist_cdf, [1.1,2.3,0.5])

In [ ]:


dist_cdf(xx::Real)::Real=DistributionsAD.pdf(DistributionsAD.Beta(1.0,2.0),xx)
dist_cdfD(xx::Real)::Real=Distributions.cdf(Distributions.Beta(1.0,2.0),xx)
dist_cdf(0.1)

ForwardDiff.derivative(dist_cdf, 0.2)

In [ ]:
using Zygote, DistributionsAD

In [ ]:
dist_cdf(x)=DistributionsAD.logpdf(DistributionsAD.Beta(1.0,2.0),x)
Zygote.gradient(dist_cdf, 0.5)

In [ ]:
z = Variable(M,Positive()) 
problem = minimize(sum(h₀ᵀA*z))
problem.constraints = [
                        z>=lowA,z<=uppA,
                        hᵀA*z==qA,
                       ]
Convex.solve!(problem,  SCS.Optimizer(verbose=true, eps=1e-10,linear_solver=SCS.DirectSolver), warmstart=false)
zConvex = evaluate(z)

λ=0.1;
bc = [BilinearConstraint(transpose(z),HA,z,0.0,X=-transpose(zsol),Y=-zsol,λ=λ), [BilinearConstraint(transpose(z),H₄A[i],z,h₄₄₄ᵀA[i]*z-q₄A[i],X=-transpose(zsol),Y=-zsol,λ=λ) for i=1:N]...]
bp = BilinearProblem(problem,bc)
r = solve!(bp,() -> COSMO.Optimizer(verbose=true),iterations=2);

# Convex.solve!(problem,  ProxSDP.Optimizer())
# Convex.solve!(problem,  CSDP.Optimizer())
# Convex.solve!(problem,  Tulip.Optimizer())

zMC = evaluate(z);

# using sparse matrices
z = Variable(M,Positive()) 
problem = minimize(sum(h₀ᵀ*z))
problem.constraints = [
                        z>=low,z<=upp,
                        hᵀ*z==q,
                       ]
Convex.solve!(problem,  SCS.Optimizer(verbose=true, eps=1e-10,linear_solver=SCS.DirectSolver), warmstart=false)
zConvex = evaluate(z)



λ=0.1;
bc = [BilinearConstraint(transpose(z),H,z,0.0,X=-transpose(zConvex),Y=-zConvex,λ=λ), [BilinearConstraint(transpose(z),H₄[i],z,h₄₄₄ᵀ[i]*z-q₄[i],X=-transpose(zConvex),Y=-zConvex,λ=λ) for i=1:N]...]
bp = BilinearProblem(problem,bc)
r = solve!(bp,() -> COSMO.Optimizer(verbose=true),iterations=1);

# Convex.solve!(problem,  ProxSDP.Optimizer())
# Convex.solve!(problem,  CSDP.Optimizer())
# Convex.solve!(problem,  Tulip.Optimizer())

zMC = evaluate(z);